In [22]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta 

In [25]:
df_parts = pd.read_csv("../Datas/df_parts_an_c_paul.csv")

In [24]:
df_parts.loc[df_parts["part_num"] == "6141"]

,inventory_id,part_num,color_id,quantity,is_spare,name_part,part_cat_id,part_material,name_color,rgb,name_cat,version,set_num,name_sets,year,theme_id,name_univ
19,14705,6141,71,2,f,Plate Round 1 x 1 with Solid Stud,21,Plastic,Light Bluish Gray,A0A5A9,Plates Round Curved and Dishes,1,8059-1,Seabed Scavenger,2010.0,315.0,Atlantis
20,14705,6141,71,1,t,Plate Round 1 x 1 with Solid Stud,21,Plastic,Light Bluish Gray,A0A5A9,Plates Round Curved and Dishes,1,8059-1,Seabed Scavenger,2010.0,315.0,Atlantis
257,2617,6141,57,1,t,Plate Round 1 x 1 with Solid Stud,21,Plastic,Trans-Neon Orange,FF800D,Plates Round Curved and Dishes,1,8758-1,Tower of Toa,2005.0,341.0,Playsets
258,2617,6141,57,2,f,Plate Round 1 x 1 with Solid Stud,21,Plastic,Trans-Neon Orange,FF800D,Plates Round Curved and Dishes,1,8758-1,Tower of Toa,2005.0,341.0,Playsets
1049,23285,6141,57,1,t,Plate Round 1 x 1 with Solid Stud,21,Plastic,Trans-Neon Orange,FF800D,Plates Round Curved and Dishes,2,8758-1,Tower of Toa,2005.0,341.0,Playsets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047955,16480,6141,80,1,t,Plate Round 1 x 1 with Solid Stud,21,Plastic,Metallic Silver,A5A9B4,Plates Round Curved and Dishes,1,30603-1,Batman Classic TV Series - Mr. Freeze,2016.0,697.0,Batman
1047969,36005,6141,35,1,f,Plate Round 1 x 1 with Solid Stud,21,Plastic,Trans-Bright Green,D9E4A7,Plates Round Curved and Dishes,1,211903-1,Superman and Kryptonite,2019.0,701.0,Superman
1047974,104757,6141,35,2,f,Plate Round 1 x 1 with Solid Stud,21,Plastic,Trans-Bright Green,D9E4A7,Plates Round Curved and Dishes,1,fig-011567,NaN,NaN,NaN,NaN
1047980,118958,6141,35,1,f,Plate Round 1 x 1 with Solid Stud,21,Plastic,Trans-Bright Green,D9E4A7,Plates Round Curved and Dishes,1,71032-11,Space Creature,2022.0,718.0,Series 22 Minifigures


In [26]:
df_parts["name_part"].nunique()

50177

In [27]:
df_parts["part_material"].value_counts(normalize = True, ascending=True).round(4)*100

Foam                 0.01
Metal                0.01
Flexible Plastic     0.02
Cloth                0.17
Cardboard/Paper      0.20
Rubber               0.72
Plastic             98.87
Name: part_material, dtype: float64

In [19]:
display(df_parts.groupby("part_material")["year"].value_counts(ascending=True).unstack())

year,1949.0,1950.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
part_material,,,,,,,,,,,,,,,,,,,,,
Cardboard/Paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.0,26.0,263.0,474.0,121.0,13.0,165.0,12.0,7.0,1.0
Cloth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,43.0,28.0,36.0,31.0,26.0,44.0,51.0,15.0,28.0
Flexible Plastic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,15.0,22.0,16.0,24.0,14.0,26.0,23.0
Foam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
Metal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,2.0,2.0,NaN,5.0,NaN,NaN,NaN,NaN,1.0
Plastic,171.0,22.0,26.0,109.0,243.0,152.0,285.0,653.0,18.0,70.0,...,42993.0,50016.0,51166.0,56832.0,58577.0,64055.0,72254.0,79004.0,75822.0,69449.0
Rubber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,336.0,323.0,276.0,316.0,284.0,312.0,335.0,352.0,319.0,239.0


In [20]:
df_parts.groupby("year")["part_num"].count()

year
1949.0      171
1950.0       22
1953.0       26
1954.0      109
1955.0      243
          ...  
2019.0    64422
2020.0    72822
2021.0    79433
2022.0    76189
2023.0    69741
Name: part_num, Length: 73, dtype: int64

In [21]:
df_spare = df_parts.groupby("year")["is_spare"].value_counts().unstack().fillna(0)
df_spare["% de pièce de rechange"] = ((df_spare["t"]/(df_spare["f"]+df_spare["t"]))*100).round(2)
display(df_spare)

is_spare,f,t,% de pièce de rechange
year,,,
1949.0,171.0,0.0,0.00
1950.0,22.0,0.0,0.00
1953.0,26.0,0.0,0.00
1954.0,109.0,0.0,0.00
1955.0,241.0,2.0,0.82
...,...,...,...
2019.0,58736.0,5686.0,8.83
2020.0,66154.0,6668.0,9.16
2021.0,71581.0,7852.0,9.89


In [84]:
df_most_used_alltime = df_parts.groupby("name_part")["quantity"].sum()
df_most_used.sort_values(ascending=False, inplace=True)
df_most_used.head(20)

name_part
Plate 1 x 2                                                     125093
Plate Round 1 x 1 with Solid Stud                               116947
Brick 1 x 2                                                      99012
Plate 1 x 1                                                      90075
Brick 1 x 1                                                      74783
Technic Pin with Friction Ridges Lengthwise and Center Slots     68212
Plate 1 x 4                                                      56647
Brick 2 x 2                                                      55094
Tile Round 1 x 1                                                 49828
Slope 30° 1 x 1 x 2/3 (Cheese Slope)                             49244
Tile 1 x 2 with Groove                                           48750
Plate 2 x 4                                                      46412
Brick 1 x 4                                                      45861
Plate 2 x 2                                                      43

In [95]:
df_most_used_years =df_parts.groupby(["name_part", "year"])["quantity"].sum().unstack()
print(df_most_used_years.idxmax()[1949])

Brick 2 x 2 without Bottom Tubes, Slotted (with 1 slot)


import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np

# Créez des données de démonstration
x = np.array([1, 2, 3, 4, 5])
y = np.array([10, 15, 7, 12, 8])

# Créez une figure et un axe
fig, ax = plt.subplots()

# Tracez le nuage de points
ax.scatter(x, y)

# Chemin vers l'image que vous souhaitez utiliser comme étiquette
image_path = 'chemin/vers/votre/image.png'

# Ajoutez des images comme étiquettes de données
for i, (xi, yi) in enumerate(zip(x, y)):
    ab = AnnotationBbox(OffsetImage(plt.imread(image_path), zoom=0.1), (xi, yi), frameon=False)
    ax.add_artist(ab)

# Affichez le graphique
plt.show()